In [1]:
import requests
import numpy as np
import pandas as pd
import os
import time
import datetime
import re
import warnings
warnings.filterwarnings('ignore')
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
import time
import numpy as np

In [2]:
col_dict = {"corp_id": str, "corp_name": str, "website": str}
corp = pd.read_excel('C:/Users/user/Downloads/corp_website_done.xlsx', dtype=col_dict)

In [3]:
corp['corp_id']

0        000036QI
1        000040BA
2        000080MI
3        000195PH
4        000259PT
           ...   
12686    98910003
12687    97413338
12688    97397885
12689    97417474
12690    97498531
Name: corp_id, Length: 12691, dtype: object

In [4]:
corp['corp_id'] = corp['corp_id'].map(lambda x:("0"*(8-len(x))+x) if len(x)<8 else x)

In [5]:
corp_id_list_total = list(corp["corp_id"])

In [6]:
corp

,corp_id,corp_name,corp_name_web,comm_name,branch_name,corp_name_web_eng,website,check,time_out
0,000036QI,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,000040BA,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
2,000080MI,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
3,000195PH,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
4,000259PT,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...
12686,98910003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12687,97413338,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12688,97397885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12689,97417474,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#selenium settings
options = Options()
options.add_argument("--disable-gpu");
chrome = webdriver.Chrome('./chromedriver',options=options)
#設定等待時間
chrome.implicitly_wait(5)
chrome.set_page_load_timeout(15)

In [18]:
corp_id_list = corp_id_list_total[6000:6500]

In [19]:
corp_id_list

['38750755',
 '42697402',
 '42773978',
 '38443032',
 '38834100',
 '41346978',
 '41426769',
 '40745104',
 '400010BR',
 '41410491',
 '41387787',
 '42828262',
 '42701721',
 '38669122',
 '39872171',
 '388637SI',
 '41216216',
 '42639473',
 '42636927',
 '38721534',
 '42497565',
 '39606792',
 '42640746',
 '40115507',
 '400444QI',
 '53110630',
 '53795653',
 '53451025',
 '53498393',
 '53080039',
 '53618799',
 '53258936',
 '53729177',
 '53093421',
 '53089073',
 '53134176',
 '53113411',
 '45664930',
 '45058879',
 '43814765',
 '45117618',
 '45693344',
 '47832103',
 '45910567',
 '45207245',
 '47686713',
 '42954865',
 '47692840',
 '45630221',
 '42990567',
 '47895227',
 '47814999',
 '45308594',
 '453516GU',
 '45503169',
 '47255947',
 '45898803',
 '45858443',
 '47885488',
 '42847227',
 '47861438',
 '47736381',
 '45894882',
 '47942729',
 '47670352',
 '47893179',
 '42926616',
 '45145115',
 '43885971',
 '47799174',
 '47674293',
 '42916856',
 '43739422',
 '47760810',
 '47282395',
 '42900138',
 '450665BR',

In [20]:
#loop
counter=1
for id in corp_id_list:
    try:
        chrome.get("https://www.twincn.com/item.aspx?no="+str(id))
        try:
            table_tr_list = chrome.find_element("class name", "table-responsive").find_elements("tag name", "tr")
            table_list = []
            for tr in table_tr_list:
                #將每一個tr的資料根據td查詢出來，返回結果為list物件
                table_td_list = tr.find_elements("tag name", "td")
                row_list = []
                for td in table_td_list:    #遍歷每一個td
                    row_list.append(td.text)    #取出表格的資料，並放入行列表裡
                table_list.append(row_list)
            for i in table_list:
                if i[0]=='網址':
                    corp.loc[corp["corp_id"]==id,"website"] = i[1]
                elif i[0]=='公司名稱':
                    corp.loc[corp["corp_id"]==id,"corp_name_web"] = i[1]
                elif i[0]=='商業名稱':
                    corp.loc[corp["corp_id"]==id,"comm_name"] = i[1] 
                elif i[0]=='分公司名稱':
                    corp.loc[corp["corp_id"]==id,"branch_name"] = i[1]                
                elif i[0]=='英文名稱':
                    corp.loc[corp["corp_id"]==id,"corp_name_web_eng"] = i[1]      
                corp.loc[corp["corp_id"]==id,"check"] = "Y"
        except:
            try:
                check_in_homepage = chrome.find_element("id", "search-btn")
                corp.loc[corp["corp_id"]==id,"check"] = "N"
            except:
                corp.loc[corp["corp_id"]==id,"time_out"] = "Y"
                print("觸發反爬蟲,目前資料處理至第{}筆,corp_id為{}".format(corp_id_list_total.index(id),id))
                break
        rand_time = np.random.randint(5)
        time.sleep(rand_time)  #強制等待
        counter = counter+1
        if counter%50==0:
            print("已完成{}筆".format(counter))
            corp.to_excel('C:/Users/user/Downloads/corp_website_done.xlsx',index=False)
    except:
        corp.loc[corp["corp_id"]==id,"time_out"] = "Y"

已完成50筆
已完成100筆
已完成150筆
已完成200筆
已完成250筆
已完成300筆
已完成350筆
已完成400筆
已完成450筆
已完成500筆


In [49]:
corp_id_list_total[4535]

'28602398'

In [21]:
corp.to_excel('C:/Users/user/Downloads/corp_website_done.xlsx',index=False)